### Data collection

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import random


import scipy
import math
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split 
from scipy.sparse.linalg import svds 
from sklearn import preprocessing 

import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lars\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
path='./Data/'

df_books=pd.read_csv(path +'filtered_books.csv') 
df_ratings=pd.read_csv(path + 'Ratings.csv') 
df_users=pd.read_csv(path + 'Users.csv') 

C:\Users\Lars\AppData\Local\Temp\ipykernel_25312\2624249305.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books=pd.read_csv(path +'filtered_books.csv')


In [8]:
df_ratings.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
df_users.describe()



,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


### Preprocessing

In [ ]:
merged_df=pd.merge(df_users,df_ratings,on='User-ID')
merged_df=pd.merge(merged_df,df_books,on='ISBN')

merged_df.shape



(851511, 12)

### Collaberative Filtering

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Load the dataset
data = pd.read_csv('data/ratings.csv')

# Ensure correct data types
data['User-ID'] = data['User-ID'].astype(int)
data['ISBN'] = data['ISBN'].astype(str)
data['Book-Rating'] = data['Book-Rating'].astype(float)

# Handle missing values if any (e.g., fill with 0 or mean rating)
data.fillna(0, inplace=True)

# Create a user-item matrix
user_item_matrix = data.pivot(index='User-ID', columns='ISBN', values='Book-Rating')

# Normalize the user-item matrix
scaler = StandardScaler()
user_item_matrix_normalized = scaler.fit_transform(user_item_matrix)

# Compute user-user similarity
user_similarity = cosine_similarity(user_item_matrix_normalized)

# Convert the similarity matrix back to a DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Function to get top N similar users
def get_top_n_similar_users(user_id, n=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).head(n+1).index.tolist()
    similar_users.remove(user_id)
    return similar_users

# Function to generate recommendations for a user
def recommend_items(user_id, n=5):
    similar_users = get_top_n_similar_users(user_id, n)
    similar_users_ratings = user_item_matrix.loc[similar_users]
    recommended_items = similar_users_ratings.mean(axis=0).sort_values(ascending=False).head(n).index.tolist()
    return recommended_items

# Example usage
user_id = 276725
recommendations = recommend_items(user_id, n=5)
print(f"Recommendations for user {user_id}: {recommendations}")

c:\Python311\Lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


IndexError: index 1495360453 is out of bounds for axis 0 with size 1495018980